In [1]:
import pandas as pd
import numpy as np
from itables import show

In [2]:
df = pd.read_parquet('../data/spend_data_large_sourcing.parquet')
show(df)
df.shape

Loading ITables v2.5.2 from the internet... (need help?)


(698411, 51)

## Remove Unneeded Columns
### Needed Columns
1. Spend_Data_Posting_Date_Fiscal_Year
2. Spend_Data_Posting_Date_Fiscal_Year_and_Quarter
3. Spend_Data_Posting_Date
4. Spend_Data_Vendor_Invoice_Date
5. Spend_Data_PO_Number
6. Spend_Data_PO_Line_Item_Number
7. Spend_Data_Vendor_Number
8. Spend_Data_Invoice_Type
9. Spend_Data_Business_Partner_Name_
10. Spend_Data_Organization
11. Spend_Data_Division_Description
12. Spend_Data_Business_Unit_Description
13. Spend_Data_Geo
14. Spend_Data_Line_Item_Text
15. Spend_Data_Vendor_Invoice_Amount_LC2_USD_
16. Spend_Data_Vendor_Name
17. Spend_Data_Sourcing_Category
18. Spend_Data_Sourcing_Portfolio

## Filter Down Spend Data - Starting Point (698,411 rows; 51 columns)
1. Remove rows where Spend_Data_Invoice_Type is 'NonPO-Related'
2. Spend_Data_Sourcing_Portfolio = 'Technology


In [3]:
#Options available in Spend_Data_Sourcing_Category
df['Spend_Data_Sourcing_Category'].unique()

array(['Legal', 'Branding', 'Non-addressable', 'Advertising', 'Financial',
       'Enterprise managed software', 'Talent acquisition contingent',
       'Network and telecom', 'Facility management', 'Office products',
       'Premises', 'Technology professional services', 'Web production',
       'Benefits', 'Business process outsourcing', 'Infrastructure',
       'Gifts & Awards', 'Business advisory services', 'Localization',
       'Teleservices', 'Training', 'Events', 'Talent acquisition perm',
       'Global Mobility', 'Hardware', 'Reseller', 'Royalties',
       'HR consulting', 'Market research', 'License compliance', 'Impact',
       'Ground Transport'], dtype=object)

In [4]:
#1 remove rows where Spend_Data_Invoice_Type is 'NonPO-Related'
df = df[df['Spend_Data_Invoice_Type'] != 'NonPO-Related']
print(df.shape)

#2 remove rows where Spend_Data_Sourcing_Portfolio is not 'Technology'
df = df[df['Spend_Data_Sourcing_Portfolio'] == 'Technology']
print(df.shape)

#3 filter down to Enterprise sourcing category
df = df[df['Spend_Data_Sourcing_Category'] == 'Enterprise managed software']
print(df.shape)


(390541, 51)
(76136, 51)
(8082, 51)


In [5]:
#count rows in each fiscal year
df['Spend_Data_Posting_Date_Fiscal_Year'].value_counts()

Spend_Data_Posting_Date_Fiscal_Year
2024    2191
2025    2166
2023    2160
2026    1565
Name: count, dtype: int64

In [6]:
#filter for 24,25,26
df = df[df['Spend_Data_Posting_Date_Fiscal_Year'].isin([2024,2025,2026])]
print(df.shape)


(5922, 51)


In [7]:
#Count unique vaules with (Vendor_Name, Spend_Data_Sourcing_Category)
print("Number of unique values with (Vendor_Number, line item text):")
df.groupby(['Spend_Data_Vendor_Number', 'Spend_Data_Line_Item_Text']).ngroups

Number of unique values with (Vendor_Number, line item text):


3226

### Count cost to run the model on the data

In [8]:
from tokens import calculate_exact_ai_cost

# Define which columns you will actually send to the AI
cols_to_use = [
    'Spend_Data_Vendor_Name', 
    'Spend_Data_Line_Item_Text', 
    'Spend_Data_GL_Account_Description' # Add others if you use them
]

# Run the calculation on your filtered df
calculate_exact_ai_cost(df, cols_to_use, model="gpt-4o-mini")

Counting tokens per row...

--- Exact Cost Analysis for 5,922 rows (gpt-4o-mini) ---
Columns used: ['Spend_Data_Vendor_Name', 'Spend_Data_Line_Item_Text', 'Spend_Data_GL_Account_Description']
Total Input Tokens:  254,312
Avg Tokens/Row:      42.9
Max Tokens/Row:      75
Est. Total Cost:     $0.2158


np.float64(0.21580679999999997)

In [9]:
# Generate a lookup table for unqiue (vendor_number, line_item_text)
lookup_table = df.groupby(['Spend_Data_Vendor_Name', 'Spend_Data_Vendor_Number', 'Spend_Data_Line_Item_Text']).size().reset_index(name='count')

#add columns parent and sub_category
lookup_table['category'] = None
lookup_table['sub_category'] = None

show(lookup_table)

print(len(lookup_table))

Loading ITables v2.5.2 from the internet... (need help?)


3226


In [11]:
from functions import categorize_lookup_table, retry_errors

# Test with 20 rows first (2 API calls)
lookup_table = categorize_lookup_table(lookup_table, batch_size=None, rows_per_call=10, delay=0.2)

# View results
show(lookup_table)

Getting access token...
✓ Token received

Processing 3206 rows in batches of 10...
Estimated API calls: 321


Batches:   3%|▎         | 9/321 [00:34<18:36,  3.58s/it]

💾 Checkpoint saved (99 rows processed)


Batches:   6%|▌         | 19/321 [01:03<14:39,  2.91s/it]

💾 Checkpoint saved (198 rows processed)


...

Batches:  99%|█████████▉| 319/321 [17:42<00:05,  2.83s/it]

💾 Checkpoint saved (3192 rows processed)


Batches: 100%|██████████| 321/321 [17:44<00:00,  3.32s/it]


✓ PROCESSING COMPLETE
📊 Results:
   Rows processed:  3,198
   Errors:          0

📈 Token Usage:
   Input tokens:    403,982
   Output tokens:   77,983
   Total tokens:    481,965

💰 Cost (GPT-4o-mini pricing):
   Input cost:      $0.0606
   Output cost:     $0.0468
   Total cost:      $0.1074

💾 Checkpoint saved to: lookup_checkpoint.parquet


Loading ITables v2.5.2 from the internet... (need help?)


## Lets match categories and subcategories to df from lookup table

step 1: add columns to df for category and sub_category

step 2: iterate through df
    - for each line see if the vendor_number and line_item_text are in the lookup table
    - if so, add the category and sub_category to the df
    - if not, add a null value to the category and sub_category

step 3: return df with category and sub_category columns



In [12]:
#make a copy of df just to be safe
df_copy = df.copy()

In [13]:
#Lets match categories and subcategories to df from lookup table

# Ensure key columns match types (e.g., both strings)
df['Spend_Data_Vendor_Number'] = df['Spend_Data_Vendor_Number'].astype(str)
df['Spend_Data_Line_Item_Text'] = df['Spend_Data_Line_Item_Text'].astype(str)

lookup_table['Spend_Data_Vendor_Number'] = lookup_table['Spend_Data_Vendor_Number'].astype(str)
lookup_table['Spend_Data_Line_Item_Text'] = lookup_table['Spend_Data_Line_Item_Text'].astype(str)



In [14]:
# Merge category and sub_category back to main df
df = df.merge(
    lookup_table[['Spend_Data_Vendor_Number', 'Spend_Data_Line_Item_Text', 'category', 'sub_category']],
    on=['Spend_Data_Vendor_Number', 'Spend_Data_Line_Item_Text'],
    how='left'
)

In [16]:
show(df)

print(df.shape)

Loading ITables v2.5.2 from the internet... (need help?)


(5922, 53)


In [17]:

# Save to data folder
df.to_parquet('../data/spend_data_categorized.parquet', index=False)

In [18]:
df.to_excel('../data/spend_data_categorized.xlsx', index=False)

## use lookup table on steves data

In [19]:

df_spend_new = pd.read_csv('../data/FY23-FY26 Enterprise Managed Software Spend.csv')

#add columns for category and sub_category
df_spend_new['category'] = None
df_spend_new['sub_category'] = None


In [20]:
#merge
df_spend_new = df_spend_new.merge(
    lookup_table[['Spend_Data_Vendor_Number', 'Spend_Data_Line_Item_Text', 'category', 'sub_category']],
    left_on=['Vendor Number', 'Line Item Text'],
    right_on=['Spend_Data_Vendor_Number', 'Spend_Data_Line_Item_Text'],
    how='left'
)

show(df_spend_new)


Loading ITables v2.5.2 from the internet... (need help?)
